In [1]:
# Load duckdb, which lets us efficiently load large files
import duckdb

# Load pandas, which lets us manipulate dataframes
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True

%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Allow named parameters (python variables) in SQL cells
%config SqlMagic.named_parameters=True

# Connect jupysql to DuckDB using a SQLAlchemy-style connection string. Either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:

There's a new jupysql version available (0.10.14), you're running 0.10.10. To upgrade: pip install jupysql --upgrade
Deploy FastAPI apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


/Users/me/projects/american-community-survey/.venv/lib/python3.10/site-packages/sql/traits.py:20: FutureWarning: named_parameters: boolean values are now deprecated. Value True will be treated as "enabled". 
Please use a valid option: "warn", "enabled", or "disabled". 
For more information, see the docs: https://jupysql.ploomber.io/en/latest/api/configuration.html#named-parameters
  warnings.warn(


In [3]:
%%sql
SELECT COUNT(*) FROM '~/data/american_community_survey/*housing*2023.parquet'

,count_star()
0,3248590


In [4]:
%%sql
SELECT COUNT(*) FROM '~/data/american_community_survey/*housing*2023.parquet'

,count_star()
0,3248590


In [7]:
%%sql

SELECT 
    'Record Type' AS column_name, 
    COUNT(*) AS total_count,
    COUNT(*) FILTER (WHERE "Record Type" IS NOT NULL) AS non_missing_count,
    COUNT(*) FILTER (WHERE "Record Type" IS NULL) AS missing_count
FROM (
    SELECT * FROM read_parquet('/Users/me/data/american_community_survey/acs_pums_individual_people_united_states_first_tranche_2023.parquet')
    UNION ALL
    SELECT * FROM read_parquet('/Users/me/data/american_community_survey/acs_pums_individual_people_united_states_second_tranche_2023.parquet')
)

UNION ALL

SELECT 
    'Housing unit/GQ person serial number' AS column_name, 
    COUNT(*) AS total_count,
    COUNT(*) FILTER (WHERE "Housing unit/GQ person serial number" IS NOT NULL) AS non_missing_count,
    COUNT(*) FILTER (WHERE "Housing unit/GQ person serial number" IS NULL) AS missing_count
FROM (
    SELECT * FROM read_parquet('/Users/me/data/american_community_survey/acs_pums_individual_people_united_states_first_tranche_2023.parquet')
    UNION ALL
    SELECT * FROM read_parquet('/Users/me/data/american_community_survey/acs_pums_individual_people_united_states_second_tranche_2023.parquet')
)

UNION ALL

SELECT 
    'Division code based on 2010 Census definitions Division code based on 2020 Census definitions' AS column_name, 
    COUNT(*) AS total_count,
    COUNT(*) FILTER (WHERE "Division code based on 2010 Census definitions Division code based on 2020 Census definitions" IS NOT NULL) AS non_missing_count,
    COUNT(*) FILTER (WHERE "Division code based on 2010 Census definitions Division code based on 2020 Census definitions" IS NULL) AS missing_count
FROM (
    SELECT * FROM read_parquet('/Users/me/data/american_community_survey/acs_pums_individual_people_united_states_first_tranche_2023.parquet')
    UNION ALL
    SELECT * FROM read_parquet('/Users/me/data/american_community_survey/acs_pums_individual_people_united_states_second_tranche_2023.parquet')
)

-- Continue this pattern for all remaining columns...

UNION ALL

SELECT 
    'Person''s Weight replicate 80' AS column_name, 
    COUNT(*) AS total_count,
    COUNT(*) FILTER (WHERE "Person's Weight replicate 80" IS NOT NULL) AS non_missing_count,
    COUNT(*) FILTER (WHERE "Person's Weight replicate 80" IS NULL) AS missing_count
FROM (
    SELECT * FROM read_parquet('/Users/me/data/american_community_survey/acs_pums_individual_people_united_states_first_tranche_2023.parquet')
    UNION ALL
    SELECT * FROM read_parquet('/Users/me/data/american_community_survey/acs_pums_individual_people_united_states_second_tranche_2023.parquet')
)

ORDER BY non_missing_count DESC;

,column_name,total_count,non_missing_count,missing_count
0,Division code based on 2010 Census definitions...,3405809,3405809,0
1,Record Type,3405809,3405809,0
2,Housing unit/GQ person serial number,3405809,3405809,0
3,Person's Weight replicate 80,3405809,3405809,0


In [9]:
import pandas as pd
import os

# File paths
file_paths = [
    '/Users/me/data/american_community_survey/acs_pums_individual_people_united_states_first_tranche_2023.parquet',
    '/Users/me/data/american_community_survey/acs_pums_individual_people_united_states_second_tranche_2023.parquet'
]

# Function to count non-missing entries
def count_non_missing(df):
    return df.notna().sum()

# Initialize an empty list to store counts
all_counts = []

# Process each file
for file_path in file_paths:
    if os.path.exists(file_path):
        # Read the parquet file
        df = pd.read_parquet(file_path)
        
        # Count non-missing entries
        counts = count_non_missing(df)
        
        # Append to the list
        all_counts.append(counts)

# Combine counts if we have multiple files
if len(all_counts) > 1:
    total_counts = sum(all_counts)
else:
    total_counts = all_counts[0]

# Create a dataframe with the results
result_df = pd.DataFrame({
    'Column': total_counts.index,
    'Non-Missing Count': total_counts.values
})

# Sort by count in descending order
result_df = result_df.sort_values('Non-Missing Count', ascending=False)

# Display the results
print(result_df)

# Optionally, save to a CSV file
# result_df.to_csv('acs_column_counts.csv', index=False)

                                                Column  Non-Missing Count
0                                          Record Type            3405809
189      Public health coverage recode allocation flag            3405809
195                  School enrollment allocation flag            3405809
194                  Highest education allocation flag            3405809
193                    Grade attending allocation flag            3405809
..                                                 ...                ...
125                             Subfamily relationship              97471
24          Grandparents responsible for grandchildren              68031
17   Veteran service-connected disability rating (p...              59439
23        Length of time responsible for grandchildren              21604
111                       Recoded detailed race code_1                  0

[286 rows x 2 columns]


In [12]:
from IPython.display import display

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

result_df

,Column,Non-Missing Count
0,Record Type,3405809
189,Public health coverage recode allocation flag,3405809
195,School enrollment allocation flag,3405809
194,Highest education allocation flag,3405809
193,Grade attending allocation flag,3405809
192,Retirement income allocation flag,3405809
191,Relationship allocation flag,3405809
190,Detailed race allocation flag,3405809
188,Private health insurance coverage recode alloc...,3405809
197,Sex allocation flag,3405809
